# `qubit_discovery` Overview

This notebook will guide you through using the core features of the `qubit_discovery` module, and show you how to put them together to optimize a fluxonium circuit using gradient descent. 

Much of the heavy lifting here is done by [`SQcircuit`](https://sqcircuit.org/), a package for simulating superconducting circuits and calculating gradients using the [`PyTorch`](https://pytorch.org/) engine. This module provides a set of utilities to quickly begin optimization. Before reading this tutorial, familiarize yourself with the basic features of `SQcircuit` [here](https://docs.sqcircuit.org/).

In [1]:
import numpy as np
import qubit_discovery as qd

/Users/sbonkowsky/anaconda3/lib/python3.10/site-packages/qutip/__init__.py:110: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


ImportError: cannot import name 'mplDeprecation' from 'matplotlib._api.deprecation' (/Users/sbonkowsky/anaconda3/lib/python3.10/site-packages/matplotlib/_api/deprecation.py)

## Circuit utilities

### Sampling

Optimization begins by choosing a random starting point. The `CircuitSampler` class makes this easy to do based on a range of element values you want to sample between. To match the paper [TODO], let's set the allowable element ranges to:

| Element type       | Minimum value | Maximum value |
|--------------------|---------------|---------------|
| Capacitor          | 1 fF          | 12 pF         |
| Inductor           | 1 fH          | 5 mH          |
| Josephson junction | 1 GHz         | 100 GHz       |

Currently `CircuitSampler` only supports circuits with a single inductive loop. However, you can optimize over that too by passing in an allowed flux range $\subset [0, 2\pi]$.

In [5]:
from qubit_discovery.optimization import CircuitSampler

# Set up a sampler
sampler = CircuitSampler(
    capacitor_range=[1e-15, 12e-12],    # in F
    inductor_range=[1e-15, 5e-6],       # in H
    junction_range=[1e9, 100e9],        # in Hz
    flux_range=[0, 2 * np.pi]             
)

The topology of the circuit to be sampled is specified by string called a "**circuit code**." In the case of single-loop circuits, this is a list of the inductive elements present in the loop (`'J'` for Josephson junction and `'L`' for inductor); a more detailed description can be found in the paper [TODO].

For instance, a fluxonium has the circuit code `'JL'`.

In [10]:
# Randomly sample a circuit from the element ranges above…
fluxonium = sampler.sample_circuit_code('JL')
# …and print out the description.
fluxonium.description()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
import torch
def target_spectrum(circuit):
    MY_SPECTRUM = torch.tensor([0.014, 2.974, 0.194])

    circuit_spectrum = circuit.efreqs[1:4] - circuit.efreqs[0]
    loss = torch.mean(torch.abs(MY_SPECTRUM - circuit_spectrum)/MY_SPECTRUM[0])

    return loss, circuit_spectrum[0]

fluxonium.set_trunc_nums([120])
_ = fluxonium.diag(10)
target_spectrum(fluxonium)

set_trunc_nums called
diag called


(tensor(448048.1074, dtype=torch.float64), 3136.8671822567717)

### Truncating circuits

## Constructing a loss function

In [5]:
qd.losses.get_all_metrics()
help(qd.losses.ALL_FUNCTIONS['number_of_gates'])

Help on function number_of_gates_loss in module qubit_discovery.losses.loss:

number_of_gates_loss(circuit: SQcircuit.circuit.Circuit) -> Tuple[Union[float, torch.Tensor], Union[float, torch.Tensor]]
    Applies a loss ``1/N`` to the number of single-qubit gates ``N`` for the
    circuit. See ``functions.number_of_gates`` for details on how the number
    of gates is calculated.
    
    Parameters
    ----------
        circuit:
            A ``Circuit`` object specifying the qubit.
    
    Returns
    ----------
        loss:
            The loss ``1/N``.
        N:
            The number of single-qubit gates.



## Performing optimization